目的：
1. 

In [4]:
from scipy.stats import pearsonr
import pandas as pd

In [5]:
data = pd.read_csv('../../db/purged/data.csv')
data.head()

,序号,经济毒性分级,经济健康总分,性别,居住地,医疗费用承担方式,治疗方式,有无远处转移,改变生活方式,不得不动用储蓄,...,生活状况（PWB）,社会/家庭状况（SWB）,情感状况（EWB）,功能状态（FWB）,肺癌相关症状（LCS）,心理弹性总分,家庭沟通和问题解决（FCPS）,利用社会资源（USR）,保持积极的前景（MPO）,家庭弹性总分
0,1,2,8,1,1,1,2,1,1,1,...,13,17,8,16,16,19,38,6,9,53
1,2,2,10,2,2,1,2,0,1,1,...,21,21,12,13,18,24,38,11,7,56
2,3,2,15,1,2,1,2,1,1,1,...,2,22,6,26,20,32,38,6,9,53
3,4,2,8,2,1,1,1,0,1,1,...,15,15,10,4,21,9,54,12,14,80
4,5,0,27,1,2,2,2,0,0,0,...,5,14,5,12,14,27,54,8,11,73


In [8]:

# 定义目标变量 "经济健康总分" 和所有自变量
X = data.drop(columns=['经济健康总分', '序号', '经济毒性分级'])  # 根据需要调整，排除无关变量
target = data['经济健康总分']

# 创建 DataFrame 用于存储特征、相关性系数和 p 值
univariate_corr_results = pd.DataFrame(columns=['Feature', 'Correlation Coefficient', 'P-Value'])

# 计算每个特征与目标变量的相关性系数和 p 值
for feature in X.columns:
    corr_coeff, p_value = pearsonr(data[feature], target)
    row = pd.DataFrame({
        'Feature': [feature],
        'Correlation Coefficient': [corr_coeff],
        'P-Value': [p_value]
    })
    univariate_corr_results = pd.concat([univariate_corr_results, row], ignore_index=True)

# 按 p 值排序结果
univariate_corr_results_sorted = univariate_corr_results.sort_values(by='P-Value').reset_index(drop=True)

# 显示结果
print(univariate_corr_results_sorted)

# 可选：将结果保存为 CSV 文件
univariate_corr_results_sorted.to_csv("univariate_correlation_results.csv", index=False)


       Feature  Correlation Coefficient       P-Value
0         支付不起                -0.277346  7.137346e-09
1    生活状况（PWB）                -0.272495  1.328166e-08
2      向他人寻求帮助                -0.259147  6.895997e-08
3       改变医疗计划                -0.257934  7.974047e-08
4         主观支持                 0.240620  5.852527e-07
..         ...                      ...           ...
58      其中自付费用                 0.027011  5.804837e-01
59        民政救助                -0.014446  7.675747e-01
60      交通费用总计                -0.011692  8.109552e-01
61      申请免费药物                 0.005267  9.141876e-01
62  减少生活必需品的消费                 0.004508  9.265270e-01

[63 rows x 3 columns]


C:\Users\dell\AppData\Local\Temp\ipykernel_10804\357948001.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  univariate_corr_results = pd.concat([univariate_corr_results, row], ignore_index=True)


In [9]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler

# 标准化数据
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 使用 LassoCV 进行交叉验证，选择最佳的 alpha 参数
lasso = LassoCV(cv=5, random_state=42).fit(X_scaled, target)

# 提取非零系数的特征
lasso_coef = pd.Series(lasso.coef_, index=X.columns)
selected_features = lasso_coef[lasso_coef != 0].index

# 输出重要特征
print("Selected features:", selected_features)
print("Lasso coefficients:", lasso_coef[lasso_coef != 0])


Selected features: Index(['改变生活方式', '使用信用卡贷款', '增加工作时间', '放弃休闲娱乐', '改变医疗计划', '向他人寻求帮助',
       '寻求慈善机构或公共援助', '支付不起', '职业', '目前就业状况', '报销比例', '家庭每月人均收入', '病情分期',
       '住院数', '医院总费用', '外购药费用', '营养费（共计）', '客观支持', '主观支持', '生活状况（PWB）',
       '社会/家庭状况（SWB）', '利用社会资源（USR）'],
      dtype='object')
Lasso coefficients: 改变生活方式         -0.099419
使用信用卡贷款        -0.491381
增加工作时间         -0.086330
放弃休闲娱乐         -0.463802
改变医疗计划         -0.603671
向他人寻求帮助        -0.714108
寻求慈善机构或公共援助     0.538246
支付不起           -0.752937
职业             -0.140438
目前就业状况          0.218518
报销比例            0.354550
家庭每月人均收入        0.204773
病情分期           -0.202218
住院数            -0.268955
医院总费用          -0.011300
外购药费用          -0.050930
营养费（共计）        -0.050422
客观支持           -0.106223
主观支持            0.724399
生活状况（PWB）      -0.425364
社会/家庭状况（SWB）   -0.092722
利用社会资源（USR）    -0.062811
dtype: float64


In [10]:

# 标准化数据
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 使用 LassoCV 进行交叉验证，选择最佳的 alpha 参数
lasso = LassoCV(cv=10, random_state=42).fit(X_scaled, target)

# 提取非零系数的特征
lasso_coef = pd.Series(lasso.coef_, index=X.columns)
selected_features = lasso_coef[lasso_coef != 0].index

# 输出重要特征
print("Selected features:", selected_features)
print("Lasso coefficients:", lasso_coef[lasso_coef != 0])


Selected features: Index(['改变生活方式', '使用信用卡贷款', '增加工作时间', '放弃休闲娱乐', '改变医疗计划', '向他人寻求帮助',
       '寻求慈善机构或公共援助', '大病救助', '支付不起', '职业', '目前就业状况', '报销比例', '家庭每月人均收入',
       '病情分期', '住院数', '医院总费用', '外购药费用', '营养费（共计）', '客观支持', '主观支持', '生活状况（PWB）',
       '社会/家庭状况（SWB）', '利用社会资源（USR）'],
      dtype='object')
Lasso coefficients: 改变生活方式         -0.106839
使用信用卡贷款        -0.504824
增加工作时间         -0.089782
放弃休闲娱乐         -0.468339
改变医疗计划         -0.609922
向他人寻求帮助        -0.731675
寻求慈善机构或公共援助     0.564074
大病救助           -0.007958
支付不起           -0.760028
职业             -0.154283
目前就业状况          0.221137
报销比例            0.359373
家庭每月人均收入        0.210596
病情分期           -0.221906
住院数            -0.295767
医院总费用          -0.001277
外购药费用          -0.067070
营养费（共计）        -0.058039
客观支持           -0.110043
主观支持            0.741288
生活状况（PWB）      -0.418671
社会/家庭状况（SWB）   -0.120242
利用社会资源（USR）    -0.076684
dtype: float64


In [14]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

target_classes = data['经济毒性分级']
# 假设 X 是原始的 DataFrame，X_scaled 是标准化后的 NumPy 数组，selected_features 是筛选出的特征名称列表
# 将标准化后的数组转为 DataFrame，并指定列名
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# 选择 Lasso 筛选出的变量
X_selected = X_scaled_df[selected_features]  

# 构建 Logistic 回归模型
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_selected, target_classes)

# 交叉验证评估模型
cv_scores = cross_val_score(log_reg, X_selected, target, cv=5, scoring='accuracy')
print("Cross-validation scores:", cv_scores)
print("Average cross-validation score:", cv_scores.mean())

# 查看各特征的回归系数和显著性
print("Logistic regression coefficients:", log_reg.coef_)
print("Classification report:\n", classification_report(target, log_reg.predict(X_selected)))


c:\Coding\AustraliaArticle\.venv\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Cross-validation scores: [0.08235294 0.0952381  0.10714286 0.04761905 0.05952381]
Average cross-validation score: 0.07837535014005602
Logistic regression coefficients: [[-1.86134015e+00 -1.13347847e-01 -1.02678302e-01 -3.81101432e-01
  -7.79104679e-01 -1.57839608e-01 -8.26955542e-02 -7.03020530e-03
  -1.20537639e-01  2.94702454e-02  7.01401347e-02  9.41095065e-02
  -3.49011202e-02 -3.19045114e-02  1.09087101e-02 -2.70986648e-01
  -3.39538691e-01  2.57100486e-02 -2.40973158e-01  1.27632236e-01
  -4.08420602e-01  8.99192379e-02 -6.62414425e-02]
 [ 8.18849699e-01 -6.95982228e-01 -9.09586146e-02 -2.34683892e-02
  -8.40033549e-01  1.66275556e-01 -6.48104942e-01 -4.07709001e-02
  -2.98017889e-01 -5.66487223e-01 -3.89956340e-02  4.37928881e-02
  -5.54103438e-02  1.30399591e-01 -4.51895552e-01 -4.53892160e-02
  -5.88623056e-01 -1.11691434e+00 -7.31583323e-01  1.71218021e-01
  -3.64787484e-01  2.22928655e-01  2.91362227e-01]
 [ 1.10511191e+00 -1.32835335e+00 -5.76526282e-02 -4.59418705e-02
  -3

c:\Coding\AustraliaArticle\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Coding\AustraliaArticle\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Coding\AustraliaArticle\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Coding\AustraliaArtic

0       8
1      10
2      15
3       8
4      27
       ..
416    24
417    27
418    21
419    20
420    20
Name: 经济健康总分, Length: 421, dtype: int64